# Unidade 01

- **Aluno**: Carmem Stefanie da Silva Cavalcante



**Avisos**


1. Todas as questões devem ser respondidas através de códigos escritos em linguagem Python 3 (**verificar o tipo do runtime**) e no espaço destinado para esse fim. Criem células Markdown para destacar quando inicia e termina as questões. 
2. O arquivo da base de dados não deve ser alterado. 

## Base de Dados



A base de dados contem informações sobre a estimativa da população dos municípios brasileiros. Os dados foram extraídos do [site do IBGE](https://downloads.ibge.gov.br/downloads_estatisticas.htm#). A base contem as seguintes colunas:

- **UF**: abreviatura do estado.
- **COD. UF**: código do estado de acordo com nomenclatura própria do IBGE
- **COD. MUNIC**: código do município de acordo com nomenclatura própria do IBGE
- **NOME DO MUNICÍPIO**: nome do município
- **POPULAÇÃO ESTIMADA**: população estimada para um determinado município

## Questão 01 (2.0 pontos)

1. (0.5 ponto) Qual a quantidade total de dados faltantes na base de dados?
2. (0.5 ponto) Quantos estados estão presentes na base de dados? Dados faltantes não são considerados valores válidos?
3. (0.5 ponto) Quantos municípios estão presentes na base de dados? Dados faltantes não são considerados valores válidos?
4. (0.25 ponto) Qual o tamanho da base de dados (em KB)?
5. (0.25 ponto) Elimine as linhas da base que contenham dados faltantes. 


In [0]:
import pandas as pd
ibge_data = pd.read_csv("populacao.csv")
ibge_data.head()

In [0]:
#1
ibge_data.isnull().sum()

UF                    2
COD. UF               2
COD. MUNIC            2
NOME DO MUNICÍPIO     2
POPULAÇÃO ESTIMADA    2
dtype: int64

In [0]:
#2
ibge_notnull = ibge_data.loc[ibge_data["UF"].notnull()]
len(ibge_notnull["UF"].unique())


27

In [0]:
#3
ibge_notnull = ibge_data.loc[ibge_data["COD. MUNIC"].notnull()]
state_and_city = ibge_notnull.loc[:,["UF","NOME DO MUNICÍPIO"]]
len(state_and_city)

5570

In [0]:
#4
ibge_data.info()

#A base de dados tem 217.7+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
UF                    5570 non-null object
COD. UF               5570 non-null float64
COD. MUNIC            5570 non-null float64
NOME DO MUNICÍPIO     5570 non-null object
POPULAÇÃO ESTIMADA    5570 non-null float64
dtypes: float64(3), object(2)
memory usage: 217.7+ KB


In [0]:
#5
new_ibge = ibge_data.dropna(axis=0)

#É possível ver que antes existiam 5572 linhas, e agora existem apenas 5570 
#pois as duas linhas com valores nulos foram apagadas
new_ibge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 5 columns):
UF                    5570 non-null object
COD. UF               5570 non-null float64
COD. MUNIC            5570 non-null float64
NOME DO MUNICÍPIO     5570 non-null object
POPULAÇÃO ESTIMADA    5570 non-null float64
dtypes: float64(3), object(2)
memory usage: 261.1+ KB


## Questão 2 (2,5 pontos)

1. (0.5 ponto) Qual a população estimada do país? É supeior a 200 milhões de habitantes?
2. (1.0 ponto) Quais os 05 estados brasileiros com o maior número de municípios?
2. (1.0 ponto) Quantos e quais estados possuem uma população média superior a 60 mil habitantes e inferior a 100 mil habitantes?

In [0]:
#1
total_pop = new_ibge["POPULAÇÃO ESTIMADA"].sum()

if(total_pop > 200000000):
  print("Maior que 200 milhões")
  print(total_pop)
else:
  print("Menor que 200 milhões")

Maior que 200 milhões
207660929.0


In [0]:
#2
cities_state = new_ibge.pivot_table(index="UF", values = "NOME DO MUNICÍPIO", aggfunc="count")
cities_state.sort_values(by="NOME DO MUNICÍPIO", ascending = False).head()

,NOME DO MUNICÍPIO
UF,
MG,853
SP,645
RS,497
BA,417
PR,399


In [0]:
#3
meanpop_state = new_ibge.pivot_table(index="UF", values = "POPULAÇÃO ESTIMADA", aggfunc="mean")
between_60_and_100 = meanpop_state.loc[(meanpop_state["POPULAÇÃO ESTIMADA"] > 60000) & (meanpop_state["POPULAÇÃO ESTIMADA"] < 100000)]
between_60_and_100

,POPULAÇÃO ESTIMADA
UF,
AM,65542.16129
SP,69914.52093


## Questão 3 (2.5 pontos)



1. (2.5 pontos) Qual a porcentagem dos municípios do Nordeste de acordo com a tabela abaixo? Dica: verificar a função [pandas.series.isin()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.isin.html)

| População do Município | Classificação |
|-----------------------------|-----------------|
| 0 < pop. <= 5.000 | muito pequeno |
| 5.000 < pop. <= 10.000 | pequeno |
| 10.000 < pop. <= 30.000 | médio |
| 30.000 < pop. <= 50.000 | grande |
| 50.000 < pop. <= 100.000 | muito grande |
| 100.000 < pop. <= 500.000 | enorme |
| 500.000 < pop. <= 2.000.000 | grandes centros |
| pop. > 2.000.000 | metrópole |

In [0]:

states = new_ibge["UF"]
nordeste = states.isin(['AL', 'BA', 'CE', 'MA', 'PE', 'PI', 'PB', 'RN', 'SE'])

ibge_nordeste = new_ibge[nordeste]

ibge_nordeste["class"] = pd.cut(ibge_nordeste["POPULAÇÃO ESTIMADA"],
                  bins=[0,5000,10000,30000,50000,100000,500000,2000000,1000000000],
                  labels=["Muito pequeno","Pequeno","Médio",
                  "Grande","Muito Grande","Enorme","Grandes centros","Metrópole"])

ibge_pivot = ibge_nordeste.pivot_table(index="class",
                             values="POPULAÇÃO ESTIMADA",
                             aggfunc="count")





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexes/category.py:148: RuntimeWarning: Values and categories have different dtypes. Did you mean to use
'Categorical.from_codes(codes, categories)'?
  dtype=dtype)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexes/category.py:148: RuntimeWarning: None of the categories were found in values. Did you mean to use
'Categorical.from_codes(codes, categories)'?
  dtype=dtype)
